<a href="https://colab.research.google.com/github/MariMurotani/ColabNotebooks/blob/main/DehazeFormer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Jul 20 03:26:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   78C    P0             34W /   70W |     190MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DehazeFormer

/content/drive/MyDrive/Colab Notebooks/DehazeFormer


In [ ]:
# リポジトリをクローン
!git clone https://github.com/IDKiro/DehazeFormer.git
%cd DehazeFormer

fatal: destination path 'DehazeFormer' already exists and is not an empty directory.
/content/drive/MyDrive/Colab Notebooks/DehazeFormer/DehazeFormer


In [ ]:
# 必要なライブラリをインストール
!pip install torch torchvision
!pip install -r requirements.txt

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DehazeFormer

/content/drive/MyDrive/Colab Notebooks/DehazeFormer


In [ ]:
import sys
sys.path.append("DehazeFormer")

import torch
from torchvision import transforms
from PIL import Image
import os
import gc
import cv2
import glob
from pytorch_msssim import ssim
from models.dehazeformer import dehazeformer_s

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデルロード
model = dehazeformer_s().to(device).eval()

# state_dict のロード（キー 'state_dict' に格納されている前提）
checkpoint = torch.load("DehazeFormer/save_models/indoor/dehazeformer-s.pth", map_location=device)
state_dict = checkpoint["state_dict"]
state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
for path in glob.glob(os.path.join("inputs", "*.*")):
    filename = os.path.basename(path)

    # 画像読み込みとリサイズ
    img = Image.open(path).convert("RGB")
    if img.width > 1024:
        scale = 1024 / img.width
        img = img.resize((1024, int(img.height * scale)), Image.Resampling.LANCZOS)

    input_tensor = transform(img).unsqueeze(0).to(device)


    # 推論
    with torch.no_grad():
        output = model(input_tensor).clamp(0, 1)

    output_img = transforms.ToPILImage()(output.squeeze(0).cpu())
    output_img.save(os.path.join("outputs", filename))

    # 解放
    del input_tensor, output, output_img
    torch.cuda.empty_cache()
    gc.collect()